**Homework 1**
*Author: Jordi Torres Vallverdú*
*Date:28/10/2025*

In [64]:
using StatFiles, DataFrames

data_path = "/Users/jorditorresvallverdu/Downloads/data_assignment.dta"

export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/discrete_choice/"


main = DataFrame(load(data_path))


Row,ID,male,ability_math,ability_language,parentcollege,school,degree,dist,period
,Float32?,Int8?,Float32?,Float32?,Int8?,Float32?,Float32?,Float32?,Float32?
1,1.0,0,1.0,0.0,0,1.0,1.0,0.0,1.0
2,1.0,0,1.0,0.0,0,0.0,1.0,0.0,2.0
3,2.0,0,0.0,0.0,1,1.0,1.0,5.0,1.0
4,2.0,0,0.0,0.0,1,0.0,1.0,5.0,2.0
5,3.0,0,0.0,0.0,0,1.0,1.0,5.0,1.0
6,3.0,0,0.0,0.0,0,1.0,1.0,5.0,2.0
7,4.0,0,0.0,0.0,0,1.0,1.0,15.0,1.0
8,4.0,0,0.0,0.0,0,1.0,1.0,15.0,2.0
9,5.0,0,-1.0,-1.0,0,1.0,1.0,5.0,1.0


I will omit the boring part, but I will try to estimate here only the static and then try to do the dynamic also. Furthermore, in the dynamic, do I want to consider instrumental variables??-->this is leading to a ccp estimation, most likely. 

**Exercise 1**
Descriptive statistics of the main variables. 

In [65]:
using DataFrames, StatsBase

describe(main)


Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Abstract…,Real,Float64,Real,Int64,Union
1,ID,2579.5,1.0,2579.5,5158.0,0,"Union{Missing, Float32}"
2,male,0.495541,0,0.0,1,0,"Union{Missing, Int8}"
3,ability_math,0.0443971,-2.0,0.0,2.0,0,"Union{Missing, Float32}"
4,ability_language,0.0246219,-2.0,0.0,2.0,0,"Union{Missing, Float32}"
5,parentcollege,0.27879,0,0.0,1,0,"Union{Missing, Int8}"
6,school,0.743505,0.0,1.0,1.0,0,"Union{Missing, Float32}"
7,degree,0.777821,0.0,1.0,1.0,0,"Union{Missing, Float32}"
8,dist,8.51978,0.0,10.0,25.0,0,"Union{Missing, Float32}"
9,period,1.5,1.0,1.5,2.0,0,"Union{Missing, Float32}"


In [66]:
#Plot some distributions of the ability and math by whether students go or not to college. 

In [67]:
using DataFrames, StatsPlots, FilePathsBase

types_ability = [:ability_math, :ability_language]
types_school = [:school, :degree] ### This is to include as an array, similar to c() in R. The : saves just the names ¡, this is why afterwards we need to actually call the vector by using the function cols(c) in the DataFrame Package. 
for i in types_ability
    for j in types_school

        plot_title = "Distribution of $(i) by $(j) Attendance" #added tric
        export_path = joinpath(export_dir, "plot_$(i)_by_$(j).png")


   p= @df main histogram(
        cols(i),
        group = cols(j),         ##revise why I need cols
        fill = (0.3, :auto),      
        linewidth = 2,             
        legend = :topright,
        xlabel = string(i),
        ylabel = "Density",
        title = plot_title,
        grid = true,
        palette = :Dark2_3,       
        normalize =:pdf 
    )
    savefig(p, export_path)
    println("✅ Plot saved to: ", export_path)
    end 

end 

✅ Plot saved to: /Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/discrete_choice/plot_ability_math_by_school.png
✅ Plot saved to: /Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/discrete_choice/plot_ability_math_by_degree.png
✅ Plot saved to: /Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/discrete_choice/plot_ability_language_by_school.png
✅ Plot saved to: /Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/discrete_choice/plot_ability_language_by_degree.png


_Facts_ :

1. We see that there is clear sorting in ability for those that go to high school and for those that end up graduating in college. 
2. $\rightarrow$ actually, a descriptive model is also the static discrete choice model of decisions at both periods. will directly jump tp that.

**Exercise 2**

1. Revealed preference argument

We assume student i chooses option $j\in J$ $\iff$ (**As1. Revealed preference argument**)
$$
\begin{align}
U_{ij}\leq U_{ij'} \forall j'\in J \setminus j \iff \\
V_{ij}(x_i, x_ij)+ \epsilon_{ij}\leq V_{ij'}(x_i, x_ij')+ \epsilon_{ij'} \iff \\
V_{ij}(x_i, x_ij)+ \epsilon_{ij}- V_{ij'}(x_i, x_ij')+ \epsilon_{ij'}\leq 0
\end{align}
$$

Here $V_{ij}(x_i, x_ij)$ represents the observed part of the utility and $\epsilon_{ij}$ is a random shock. The observed part of the utility depends on student characteristics, such as ability in math or in language, and in i,j characteristics such as distance to high school (which is a cost measure). Crucially, we assume $\epsilon_{ij}$ is type I e.v and independent across i and j. In our example, if we take the case of the static decision of going to high school or not, this implies that the epsilons are just idiosyncratic taste shocks. (justifying a type I e.v is easier for logit with only two options, no?)-->cite Train. 

This assumption allows us to simply write the probability of observing a given option as:
$$ P_{ij}= \frac{\exp(V_{ij})}{\exp(V_{ij})+\exp(V_{ij'})} $$

Which we can normalize simply to: 

$$
P_{ij} = \frac{1}{1 + \frac{\exp(V_{ij'})}{\exp(V_{ij})}}
$$

And with a binary logit this simplifies to (normalizing the other option as 0 utility):

$$
P_{ij} = \frac{1}{1 + \exp(-V_{i})}
$$

And then we can simply write the log-likelihood as:

$$ LL(\beta)= \sum_{i}y_{i} \log(P_{i})+ (1-y_{i})\log(1-P_{i}) $$


$\rightarrow$ I can further simplify this because the log of this is simply the difference in coefficients, this is what I will use in the loglikelihood equation.

In our case, we know that this is the same as:

$$ LL(\beta)= \sum_{i}y_{i} \log(P_{i})+ (1-y_{i})\log(1-P_{i}) $$

And McFadden showed this is stricly concave, so we can use any package in Julia such as Optim to find the solution. I propose to estimate three 


An implicit assumption in this model is that the students make myopic decisions or don't consider the future when making their decision. Here we model the decision to attend highschool as just depending on student characteristics and we don't model whether they take into account future benefits of high-school in making the decision. We really can't distinguish between flow values of the utilities--> why do we need a dynamic model here? what do you want to say something about? because in a static model could we actually account for the future flow utility? it should be implicitly there?. 


**Dynamic Problem**

Now suppose instead that students consider the future in making the decision to attend high school or not. Suppose we have only two-time periods and agents consider both decisions a $t_1$ and $t_2$. In $t_1$ they consider the same choice as before, but now in $t_2$ they need to decide whether they attend college or not. 

$$ v_{ijt}= u_{ijt} + \beta \mathbb{E}_{\epsilon_{ijt'},g_{it}}(u_{ijt'}) $$

This nests the problem from above when $\beta=0$. Note also that the expectation in the second period depends on the unobserved shock and the probability of graduating if attending college (state transition). This last element depends on the decision at t.

In [68]:
##Define the log-likelihood. 

#define variables 
period1= main[main.period .==1, :]

y=  period1.school
x1= period1.ability_math
x2= period1.ability_language
x3= period1.parentcollege
x4= period1.male

X= hcat(ones(nrow(period1)), x1, x2, x3, x4)
#should filter the obseravtions

5158×5 Matrix{Union{Missing, Float64}}:
 1.0   1.0   0.0  0.0  0.0
 1.0   0.0   0.0  1.0  0.0
 1.0   0.0   0.0  0.0  0.0
 1.0   0.0   0.0  0.0  0.0
 1.0  -1.0  -1.0  0.0  0.0
 1.0   1.0   2.0  1.0  0.0
 1.0  -1.0  -2.0  0.0  1.0
 1.0   0.0  -1.0  0.0  0.0
 1.0  -1.0  -1.0  0.0  0.0
 1.0   1.0   1.0  1.0  1.0
 ⋮                     
 1.0  -1.0  -1.0  0.0  1.0
 1.0   1.0   1.0  0.0  1.0
 1.0   1.0   1.0  0.0  0.0
 1.0  -1.0  -2.0  0.0  1.0
 1.0   1.0   1.0  1.0  0.0
 1.0   1.0   0.0  1.0  0.0
 1.0   2.0   1.0  1.0  0.0
 1.0   0.0   0.0  0.0  0.0
 1.0  -2.0   0.0  0.0  0.0

In [69]:
##Define the log_likelihood function for the binary case. 

function log_likelihood(β::AbstractVector, X::AbstractMatrix, y::AbstractVector)
    V= X*β
    p= @. 1/(1+exp(-V))    
    l= @. y*log(p) + (1-y)*log((1-p)) ## @ works here to make everything element wise operation, instead of putting a point everywhere

    return(-sum(l))
end

log_likelihood (generic function with 1 method)

In [70]:
##Minimize
using Optim

β0 = zeros(size(X, 2)) ##need to specify a candidate value. 
res= optimize(β->log_likelihood(β, X, y),β0, BFGS())

β_hat = Optim.minimizer(res)

Optim.minimum(res)


1296.9489476888016

_Computing Standard errors_ 
We follow these steps: 

1. Compute the Hessian $H(\beta)=$
2. $V=\hat{Var}(\hat{\beta})=inv(H(\hat{\beta}))$
3. $SE(\hat{\beta})=diag(\sqrt{.V})$

In [71]:
using ForwardDiff, LinearAlgebra

f = β -> log_likelihood(β, X, y)  
H = β -> ForwardDiff.hessian(f, β)
H_hat= H(β_hat)

V= inv(H_hat)
se= sqrt.(diag(V))


5-element Vector{Float64}:
 0.10388885523227864
 0.08071391470891635
 0.08124045667553148
 0.1899293897416697
 0.1106501556625681

In [72]:
##Now the idea is to try and create one function that computes all the models for a given input (y, X). 


using Optim, ForwardDiff, LinearAlgebra, DataFrames

function get_betas_se(y, X, varnames)

#Estimate parameters    
β0 = zeros(size(X, 2)) ##need to specify a candidate value. 
res= optimize(β->log_likelihood(β, X, y),β0, BFGS())
β_hat= Optim.minimizer(res)

#Estimate S.E
f = β -> log_likelihood(β, X, y)  # returns a scalar
H = β -> ForwardDiff.hessian(f, β)
H_hat= H(β_hat)

V= inv(H_hat)
se= sqrt.(diag(V))

#Return results in a dataframe
results_df = DataFrame(
        term = ["β$i ($name)" for (i, name) in enumerate(varnames)], ##quite nice tool to actually loop in a more efficient way over order of variables and variable names. 
        estimate = β_hat,
        std_error = se
    )

return(results_df)

    
end

varnames=["intercept", "ability_math", "ability_language", "parentcollege", "male"] ##this I need to find a better way
results= get_betas_se(y, X, varnames)

Row,term,estimate,std_error
,String,Float64,Float64
1,β1 (intercept),3.09666,0.103889
2,β2 (ability_math),0.525086,0.0807139
3,β3 (ability_language),0.706278,0.0812405
4,β4 (parentcollege),0.958382,0.189929
5,β5 (male),-0.949152,0.11065


**Explaining Graduation**

In [73]:
#now let's define the variables for the U2 model. 

period2= main[main.period .==2, :]

y_2=  period2.school
x1= period2.ability_math
x2= period2.ability_language
x3= period2.parentcollege
x4= period2.male
x5= period2.dist
X_2= hcat(ones(nrow(period2)), x1, x2, x3, x4, x5)


5158×6 Matrix{Union{Missing, Float64}}:
 1.0   1.0   0.0  0.0  0.0   0.0
 1.0   0.0   0.0  1.0  0.0   5.0
 1.0   0.0   0.0  0.0  0.0   5.0
 1.0   0.0   0.0  0.0  0.0  15.0
 1.0  -1.0  -1.0  0.0  0.0   5.0
 1.0   1.0   2.0  1.0  0.0  10.0
 1.0  -1.0  -2.0  0.0  1.0  20.0
 1.0   0.0  -1.0  0.0  0.0   5.0
 1.0  -1.0  -1.0  0.0  0.0  10.0
 1.0   1.0   1.0  1.0  1.0   0.0
 ⋮                           ⋮
 1.0  -1.0  -1.0  0.0  1.0   5.0
 1.0   1.0   1.0  0.0  1.0   5.0
 1.0   1.0   1.0  0.0  0.0   5.0
 1.0  -1.0  -2.0  0.0  1.0   5.0
 1.0   1.0   1.0  1.0  0.0  15.0
 1.0   1.0   0.0  1.0  0.0  15.0
 1.0   2.0   1.0  1.0  0.0  10.0
 1.0   0.0   0.0  0.0  0.0   5.0
 1.0  -2.0   0.0  0.0  0.0  15.0

In [74]:
##Now we use previous function to optimize over this

varnames2=varnames=["intercept", "ability_math", "ability_language", "parentcollege", "male", "distance"] ##this I need to find a better way
results2= get_betas_se(y_2, X_2, varnames2)

Row,term,estimate,std_error
,String,Float64,Float64
1,β1 (intercept),0.617624,0.0813367
2,β2 (ability_math),0.912924,0.0645824
3,β3 (ability_language),1.10502,0.065319
4,β4 (parentcollege),1.10181,0.0944021
5,β5 (male),-0.825629,0.0746677
6,β6 (distance),-0.00822581,0.0070997


#Now we can simply find the preferences in terms of cost by dividing the coefficients by the distance coefficient. However, note that the coefficient is not precisely estimated. Should we still carry on with this? Is my specification correct?

In [75]:

cost_coef=results2.estimate[6]
coef= results2.estimate
coef_normalized= coef ./cost_coef  

##How to obtain standard errors on this?


6-element Vector{Float64}:
  -75.08362161157454
 -110.98288274047157
 -134.33601609848844
 -133.94582548956757
  100.37054437514762
    1.0

*Graduation*
Finally, a simple model that predicts graduation

In [76]:

y_3=  period2.degree

varnames=["intercept", "ability_math", "ability_language", "parentcollege", "male"] ##this I need to find a better way
results3= get_betas_se(y_3, X, varnames)


Row,term,estimate,std_error
,String,Float64,Float64
1,β1 (intercept),1.86257,0.069564
2,β2 (ability_math),0.666215,0.0639793
3,β3 (ability_language),0.930481,0.065311
4,β4 (parentcollege),1.06143,0.129027
5,β5 (male),-0.744599,0.0821727


**DYNAMIC MODEL**

Let's consider the following dynamic model. At period 0, students can decide whether to go to high school or not. Conditional on them having gone to college in period 0 and having graduated, they can decide to go to higher education or not in period 1. The state variable is thus going to college in period 0 (which affects decisions in the next period).  At t=0, then the two value functions will be:
$$
\begin{align}
V_{i00}= U_{i0}^{0}(\theta_{0})+ \beta \mathbb{E}(\bar{V_{1}}(g=0)) //
V_{i10}= U_{i1}^{0} + \beta (P(G=1|X, C=1)\mathbb{E}(V_{1}(g=1))+ (1-P(G=1|X, C=1))\mathbb{E}(\bar{V_{1}}(g=0)))
\end{align}
$$

We know that student i will decide to go to high school at time 0 $\iff$
$$
\begin{align}
V_{1}\leq V_{O} \iff \\
U_{i1}^{0} + \beta (P(G=1|X, C=1)\mathbb{E}(V_{1}(g=1))+ (1-P(G=1|X, C=1))\mathbb{E}(\bar{V_{1}}(g=0)))\leq U_{i0}^{0}(\theta_{0})+ \beta \mathbb{E}(\bar{V_{1}}(g=0)) 
\end{align}
$$




Given that only differences in utilities can be identified, we can ser U(0)= 0 in both periods. Also, given that in t=2, the individual faces a static decision- no matter what shock she has, she knows she will decide optimally. We can do the following trick:
$$
\mathbb{E}(V_{1}(g=1))=\gamma + \ln (\exp(U_{1}^{1})+\exp(U_{0}^{1}))
$$

$$
\mathbb{E}(V_{1}(g=0))=U_{i0}^{0}=0 (normalized)
$$
This and the normalization of the utilities allows us to express:

$$
V_{1}\leq V_{O} \iff
\epsilon_{1}^{0}-\epsilon_{0}^{0}\leq  -U_{1}^{0} -\gamma-  \log(\exp(U_{1}^{1})+\exp(U_{0}^{1}))
$$

And thus we can code the ll: (note that we can also model P in the loglikelihood, we need it too there, the three decisions together)

$$
LL(\beta)= \frac{1}{1+\exp(-U_{1}^{0} -\gamma-  \log(\exp(U_{1}^{1})+\exp(U_{0}^{1})))}
$$

And the last element is 0, no? $\exp(U_{0}^{1}) 1$?


Second, for the loglikelihood, we need to consider:


$$
\begin{aligned}
P(S=1, G=0, C=0)
  &= P(S=1)\,\big(1 - P(G=1 \mid S=1)\big), \\[4pt]
P(S=1, G=1, C=0)
  &= P(S=1)\,P(G=1 \mid S=1)\,\big(1 - P(C=1 \mid G=1, S=1)\big), \\[4pt]
P(S=1, G=1, C=1)
  &= P(S=1)\,P(G=1 \mid S=1)\,P(C=1 \mid G=1, S=1), \\[4pt]
P(S=0, G=0, C=0)
  &= 1 - P(S=1).
\end{aligned}
$$
$$
\begin{aligned}
LL(\beta) = \sum_i \Big[ 
& S_i (1-G_i)(1-C_i) 
  \big( \log P(S_i=1) + \log (1 - P(G_i=1 \mid S_i=1)) \big) \\[6pt]
&+ S_i G_i (1-C_i) 
  \log \Big( (1 - P(C_i=1 \mid G_i=1, S_i=1)) 
              P(G_i=1 \mid S_i=1) 
              P(S_i=1) \Big) \\[6pt]
&+ S_i G_i C_i 
  \log \Big( P(C_i=1 \mid G_i=1, S_i=1) 
              P(G_i=1 \mid S_i=1) 
              P(S_i=1) \Big) \\[6pt]
&+ (1-S_i)(1-G_i)(1-C_i) 
  \log \big( 1 - P(S_i=1) \big)
\Big].
\end{aligned}
$$


We can parametrize then the probabilities by simply defining:

$$
P(S_i=1)=\frac{1}{1+\exp(-U_{1}^{0} -\gamma-  \log(\exp(U_{1}^{1})+1))}
$$
$$
P(G_i=1)=\frac{1}{1+\exp(-Z_{i}'\gamma)}
$$
$$
P(C_i=1)=\frac{1}{1+\exp(-C_{i}'\theta)}
$$

Note that we can estimate a static and control for the flow utility of the future and we should get the same results? or we care about self-selected sample..


In [78]:
using LinearAlgebra

function log_likelihood_joint(β_all, X1, X2, s, g, c)
    # unpack parameters
    k1 = size(X1, 2)
    k2 = size(X2, 2)
    k3 = size(X1, 2)

    β_1 = β_all[1:k1]
    θ   = β_all[k1+1:k1+k2]
    γ   = β_all[k1+k2+1:k1+k2+k3]
    disc= 0.95

    # college utility (used in both places)
    U_col = X2 * θ

    #  probability of going to school (dynamic)
    V1 = X1 * β_1
    γ_euler = 0.5772
    p_s = @. 1 / (1 + exp(-(V1 -γ_euler - log(exp(U_col) + 1))))

    #probability of graduating
    Z1 = X1 * γ
    p_g = @. 1 / (1 + exp(-Z1))

    #probability of going to college
    p_c = @. 1 / (1 + exp(-U_col))

    #avoid log(0)
    eps = 1e-12
    p_s = clamp.(p_s, eps, 1 - eps)
    p_g = clamp.(p_g, eps, 1 - eps)
    p_c = clamp.(p_c, eps, 1 - eps)

    #log-likelihood contributions
    l = @. s*(1-g)*(1-c)*(log(p_s) + log(1 - p_g)) +
           s*g*(1-c)*(log(1 - p_c) + log(p_g) + log(p_s)) +
           s*g*c*(log(p_c) + log(p_g) + log(p_s)) +
           (1 - s)*(1 - g)*(1 - c)*log(1 - p_s)

    return -sum(l)
end


log_likelihood_joint (generic function with 1 method)

In [79]:
#define variables 

s=  period1.school
x1= period1.ability_math
x2= period1.ability_language
x3= period1.parentcollege
x4= period1.male

X1= hcat(ones(nrow(period1)), x1, x2, x3, x4)
#should filter the obseravtions

g=  period1.degree

c=  period2.school
x1= period2.ability_math
x2= period2.ability_language
x3= period2.parentcollege
x4= period2.male
x5= period2.dist

X2= hcat(ones(nrow(period2)), x1, x2, x3, x4, x5)
#should filter the obseravtions

5158×6 Matrix{Union{Missing, Float64}}:
 1.0   1.0   0.0  0.0  0.0   0.0
 1.0   0.0   0.0  1.0  0.0   5.0
 1.0   0.0   0.0  0.0  0.0   5.0
 1.0   0.0   0.0  0.0  0.0  15.0
 1.0  -1.0  -1.0  0.0  0.0   5.0
 1.0   1.0   2.0  1.0  0.0  10.0
 1.0  -1.0  -2.0  0.0  1.0  20.0
 1.0   0.0  -1.0  0.0  0.0   5.0
 1.0  -1.0  -1.0  0.0  0.0  10.0
 1.0   1.0   1.0  1.0  1.0   0.0
 ⋮                           ⋮
 1.0  -1.0  -1.0  0.0  1.0   5.0
 1.0   1.0   1.0  0.0  1.0   5.0
 1.0   1.0   1.0  0.0  0.0   5.0
 1.0  -1.0  -2.0  0.0  1.0   5.0
 1.0   1.0   1.0  1.0  0.0  15.0
 1.0   1.0   0.0  1.0  0.0  15.0
 1.0   2.0   1.0  1.0  0.0  10.0
 1.0   0.0   0.0  0.0  0.0   5.0
 1.0  -2.0   0.0  0.0  0.0  15.0

In [ ]:
using Optim, ForwardDiff, Distributions

##this needs to be fixed, I don't know for some iterations it does not converge. Also, I don't know if I am doing it correctly. So, I need to definetely fixed this and the above results. It is also very intensive on my pc so definetely is inefficient. Try to find better solution. 
n1 = size(X1, 2)
n2 = size(X2, 2)
n3 = size(X1, 2)

ll_max= Inf
beta_top= nothing

for i in 1:100

β0 = rand(Uniform(-4, 4), n1 + n2 + n3) ##At each iteration, get new random starting values. Avoid local maximum. 

    res = optimize(
        β -> log_likelihood_joint(β, X1, X2, s, g, c),
        β0,
        BFGS();
        autodiff = :forward
    )
    

    ll= Optim.minimum(res) ##get the ll

    if ll<ll_max #keep the lowest from the whole iterations. 
        ll_max=ll
        β_hat= Optim.minimizer(res)
    end 


end 



AssertionError: AssertionError: isfinite(phi_c) && isfinite(dphi_c)

In [ ]:
using ForwardDiff, LinearAlgebra


f = β -> log_likelihood_joint(β, X1, X2, s, g, c)  # returns a scalar
H = β -> ForwardDiff.hessian(f, β)

H_hat= H(β_hat)
V= inv(H_hat)
se= sqrt.(diag(V))



16-element Vector{Float64}:
 0.10856161339686839
 0.08650761193870382
 0.08643109435693527
 0.20704432455041188
 0.1178696824385358
 0.08969227851436905
 0.07102570488912745
 0.0711219026509979
 0.10794342605183943
 0.08362565465301426
 0.0077645847052689445
 0.08121267657246803
 0.07496114474077274
 0.0764428167817595
 0.15971670406537936
 0.09714243087862189

In [ ]:
##Putting it all together nicely. 

varnames= ["intercept_1", "ability_math_1", "ability_language_1", "parentcollege_1", "male_1", "intercept_2", "ability_math_2", "ability_language_2", "parentcollege_2", "male_2", "distance_2", "intercept_col", "ability_math_col", "ability_language_col", "parentcollege_col", "male_col"]
results_df = DataFrame(
        term = ["β$i ($name)" for (i, name) in enumerate(varnames)], ##quite nice tool to actually loop in a more efficient way over order of variables and variable names. 
        estimate = β_hat,
        std_error = se
    )



Row,term,estimate,std_error
,String,Float64,Float64
1,β1 (intercept_1),5.03123,0.108562
2,β2 (ability_math_1),0.856858,0.0865076
3,β3 (ability_language_1),1.11307,0.0864311
4,β4 (parentcollege_1),1.70108,0.207044
5,β5 (male_1),-1.23328,0.11787
6,β6 (intercept_2),1.05238,0.0896923
7,β7 (ability_math_2),0.745717,0.0710257
8,β8 (ability_language_2),0.860125,0.0711219
9,β9 (parentcollege_2),0.954359,0.107943
